In [1]:
import sys
sys.path.insert(1, r'C:\Users\Kart\jupyter_projects')
import mygit as mg

In [2]:
folder=r'C:\Users\Kart\jupyter_projects\webtrade'
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
gt.push_commit(git_url,commit_msg='update')

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [3]:
gt.

SyntaxError: invalid syntax (<ipython-input-3-8057e1725f9c>, line 1)